From https://fivethirtyeight.com/features/can-you-guess-the-mystery-word/

In [481]:
word = 'aaooo'
guess = 'bbbaa'
print(score(word,guess))
word = 'aoaoo'
guess = 'bbbaa'
print(score(word,guess))

[0, 0, 0, 2, 2]
[0, 0, 0, 2, 2]


In [480]:
from random import shuffle
WORDS = [word.strip() for word in open('enable1.txt') if len(word.strip()) == 5]
SCORE_SAMPLES = True
def score(word, guess):
    """
    Returns a list of 5 positional scores, where 0 = no match, 1 = green, 2 = yellow.
    """
    match_index = [int(w == g) for w, g in zip(word, guess)]
    word_trim = [c for c, x in zip(word, match_index) if not x]
    for i, (g, x) in enumerate(zip(guess, match_index)):
        if not x and g in word_trim:
            match_index[i] = 2
            word_trim.remove(g)
    return match_index
def filter(guesses, word):
    """
    For a set of guesses and a mystery word, returns all the words
    that would produce the same scores as the mystery word.
    """
    words = WORDS[:]
    for guess in guesses:
        s = score(word, guess)
        words = [w for w in words if score(w, guess) == s]
    return words
def prob(guesses, sample_size = None):
    """
    For the set of guesses, returns the probability of guessing a random mystery word.
    """
    p = 0
    words = WORDS[:]
    if sample_size:
        shuffle(words)
        words = words[:sample_size]
    for i, word in enumerate(words):
        print(i, end = '\r')
        candidates = filter(guesses, word)
        p += 1 / sum(c[0] == word[0] for c in candidates)
    return round(p / len(words), 4)
if SCORE_SAMPLES:
    guesses1 = ['teach', 'track', 'tacks', 'tails']
    print(filter(guesses1, 'tacos'))
    print(prob(guesses1, 100))
    guesses2 = ['magic', 'maims', 'sumps', 'mosso']
    print(filter(guesses2, 'misos'))
    print(prob(guesses2, 100))

['tacos', 'tacts']
0.4589
['misos']
0.3688


In [49]:
#trying random list of words:

sample = WORDS.copy()
shuffle(sample)
random_guesses = sample[0:4]
print(random_guesses)
print(prob(random_guesses,100))

['caput', 'fluyt', 'soggy', 'hippo']
0.5419


In [482]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [483]:
df2 = pd.DataFrame(WORDS)
df = pd.DataFrame([[char for char in item] for item in WORDS],columns = [0,1,2,3,4])
# score each word based on how common each letter | position is, with minimal overlap between the words
# individual position score & total document score
score_dict = {}
variance_dict = {}
for col in df.columns:
    score_dict[col] = df[col].value_counts() / df.shape[0] # position frequency
    variance_dict[col] = np.var(score_dict[col]) # high variance means more likely to guess right, may use?
overall_dict = pd.melt(df).value.value_counts() / pd.melt(df).shape[0] # total document frequency

In [484]:
word_scores = {}
for word in WORDS:
    guess_score = 0
    pos = 0
    for letter in word:
        guess_score += score_dict[pos][letter] 
#         if letter not in word[:pos]:
#             guess_score += overall_dict[letter]**(.001)
        pos += 1
    word_scores[word] = round(guess_score,4)
    
s  = pd.Series(word_scores,index=word_scores.keys())
# s.sort_values(ascending = False,inplace=True)

In [485]:
df3 = pd.concat([df2,df,s.reset_index()[0]],axis=1)
df3.columns = ['word',0,1,2,3,4,'guess_score']
df3.sort_values(by='guess_score',ascending=False,inplace=True)
df3.reset_index(inplace=True)

guess_list = []
for i in range(4):
    guess = df3.loc[0]
    guess_list.append(guess['word'])
    df3 = df3[(df3[0] != guess[0]) & (df3[1] != guess[1]) \
            & (df3[2] != guess[2]) & (df3[3] != guess[3]) \
            & (df3[4] != guess[4])].reset_index(drop = True)
    print(guess['word'])

sores
canty
telae
liard


In [486]:
df3 = pd.concat([df2,df,s.reset_index()[0]],axis=1)
df3.columns = ['word',0,1,2,3,4,'guess_score']
df3.sort_values(by='guess_score',ascending=False,inplace=True)
df3.reset_index(inplace=True)

guess = df3.loc[0]
df4 = df3[(df3[0] != guess[0]) & (df3[1] != guess[1]) \
        & (df3[2] != guess[2]) & (df3[3] != guess[3]) \
        & (df3[4] != guess[4])].reset_index(drop = True)

In [487]:
print(guess['word'])
df4.head()

sores


,index,word,0,1,2,3,4,guess_score
0,1150,canty,c,a,n,t,y,0.4984
1,549,banty,b,a,n,t,y,0.4896
2,5314,panty,p,a,n,t,y,0.4868
3,1143,canny,c,a,n,n,y,0.4868
4,1144,canoe,c,a,n,o,e,0.4817


In [488]:
print(guess_list)
print(prob(guess_list))

['sores', 'canty', 'telae', 'liard']
0.7911
